In [2]:
import requests
import pandas as pd
import datetime
from functools import wraps
import os

In [4]:
# Decorators
def error_log(function_name):
    def inner_decorator(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            try:
                result = func(*args, **kwargs)
                return result
            except Exception as e:
                print(f"Function {function_name} encountered an error: {e}")
                
        return wrapper
    return inner_decorator

In [13]:
# @error_log('fetch_data')
def fetch_data(url):

    # call API
    r = requests.get(url)
    op = r.json()
    
    # json to data frame
    df = pd.json_normalize(op['response']['data'])
    df = df.pivot(columns="type-name", values='value', index = ['period', 'respondent'] )
    
    df.reset_index(inplace=True)
    
    # organize data as needed
    #df.drop(columns=['Day-ahead demand forecast','Total interchange'], inplace=True)
    df[['Date', 'Hour']] = df['period'].str.split('T',expand=True)
    df = df.drop('period', axis = 1)
    df.rename(columns={'respondent':'region'}, inplace=True)
    df = df[['Date','Hour','region','Demand','Net generation','Day-ahead demand forecast','Total interchange']]
    
    # segregate data based on regions
    final_dict = segregate_to_region(df)

    return final_dict

In [6]:
# @error_log('segregate_to_region')
def segregate_to_region(df):
    data = {}
    for r in regions:
        data[r] = df[df['region']==r].reset_index().drop(columns=['index'])
        if len(data[r])<96:
            temp = data_correction(r,data[r])
            temp.reset_index(inplace=True)
            temp.drop(columns=['index'],inplace=True)
            data[r] = temp
        
    return data

In [22]:
# @error_log('data_correction')
def data_correction(r,df):
    hour = ['{:02d}'.format(i) for i in range(24)]
    uq_date = df['Date'].unique().tolist()
    dfs=[]
    for dt in uq_date:
        uq_hour = df[df['Date']==dt]['Hour'].unique().tolist()
        missing_hr = [x for x in hour if x not in uq_hour]
        if len(missing_hr)!=0:
            new_df=generate_data(df[(df['Date']==dt)],missing_hr,r,dt)
#             print(r,dt,missing_hr,len(new_df))
            dfs.append(new_df)
        else:
            dfs.append(df[(df['Date']==dt)])
    print('Data Correction')
    print(uq_date)
    dfs = pd.concat(dfs)
#     print(len(dfs))
    return dfs

In [14]:
# @error_log('generate_data')
def generate_data(df,missing_hr,r,dt):
    
    demand_median = df['Demand'].median()
    net_gen_median = df['Net generation'].median()
    
    forecast_median = df['Day-ahead demand forecast'].median()
    interchange_median = df['Total interchange'].median()
    
    new_data=[]
    for hr in missing_hr:
        new_data.append((dt,hr,r,demand_median,net_gen_median,forecast_median,interchange_median))
        
    df_missing = pd.DataFrame(new_data,columns=['Date', 'Hour', 'region','Demand','Net generation','Day-ahead demand forecast','Total interchange'])
    df_new = pd.concat([df,df_missing])
    df_new = df_new.sort_values(['Date','Hour'])
    
    return df_new

In [9]:
# @error_log('get_new_dates')
def get_new_dates(start_date,end_date):
    start_date = datetime.datetime.strptime(start_date, '%Y-%m-%d')
    end_date = datetime.datetime.strptime(end_date, '%Y-%m-%d')

    st =  end_date + datetime.timedelta(days=1)
    start_date = st.strftime('%Y-%m-%d')
    end_date = (st + datetime.timedelta(days=3)).strftime('%Y-%m-%d')
    
    return start_date, end_date

In [23]:
start_date =  "2015-07-01"
end_date = "2015-07-04"
regions = ['CAL', 'CAR', 'CENT', 'FLA', 'MIDA', 'MIDW', 'NE', 'NY', 'SE', 'SW', 'TEX']
region_data_dict={}

In [24]:
while end_date<='2023-07-10':
    
    url= f"https://api.eia.gov/v2/electricity/rto/region-data/data/?\
frequency=hourly&data[0]=value&facets[respondent][]=CAL\
&facets[respondent][]=CAR&facets[respondent][]=CENT&facets[respondent][]=FLA&facets[respondent][]=MIDA\
&facets[respondent][]=MIDW&facets[respondent][]=NE&facets[respondent][]=NY&facets[respondent][]=SE&\
facets[respondent][]=SW&facets[respondent][]=TEX&\
start={start_date}T00&end={end_date}T23\
&sort[0][column]=period&sort[0][direction]=desc&offset=0&length=5000\
&api_key=2Ztw7IK10RqAv0oag9T2o2FOV8YZgRpapfTEvhwH"
    print(f'Now fetching data from {start_date} to {end_date}')
    temp=fetch_data(url)
    ct=0
    for k in temp.keys():
        if k in region_data_dict.keys():
            region_data_dict[k].append(temp[k])
            ct+=len(temp[k])
        else:
            region_data_dict[k]=[temp[k]]
            ct+=len(temp[k])

    print(f'Got demand data from date {start_date} to {end_date} and its count is {ct}')
    
    start_date,end_date = get_new_dates(start_date,end_date)

Now fetching data from 2015-07-01 to 2015-07-04
Data Correction
['2015-07-01', '2015-07-02', '2015-07-03', '2015-07-04']
Data Correction
['2015-07-01', '2015-07-02', '2015-07-03', '2015-07-04']
Data Correction
['2015-07-01', '2015-07-02', '2015-07-03', '2015-07-04']
Data Correction
['2015-07-01', '2015-07-02', '2015-07-03', '2015-07-04']
Data Correction
['2015-07-01', '2015-07-02', '2015-07-03', '2015-07-04']
Data Correction
['2015-07-01', '2015-07-02', '2015-07-03', '2015-07-04']
Data Correction
['2015-07-01', '2015-07-02', '2015-07-03', '2015-07-04']
Data Correction
['2015-07-01', '2015-07-02', '2015-07-03', '2015-07-04']
Data Correction
['2015-07-01', '2015-07-02', '2015-07-03', '2015-07-04']
Data Correction
['2015-07-01', '2015-07-02', '2015-07-03', '2015-07-04']
Data Correction
['2015-07-01', '2015-07-02', '2015-07-03', '2015-07-04']
Got demand data from date 2015-07-01 to 2015-07-04 and its count is 1056
Now fetching data from 2015-07-05 to 2015-07-08
Got demand data from date 20

/workspace/VoltWise/.venv/lib/python3.10/site-packages/numpy/lib/nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/workspace/VoltWise/.venv/lib/python3.10/site-packages/numpy/lib/nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)


Data Correction
['2018-07-01', '2018-07-02', '2018-07-03', '2018-07-04']
Got demand data from date 2018-07-01 to 2018-07-04 and its count is 1056
Now fetching data from 2018-07-05 to 2018-07-08
Got demand data from date 2018-07-05 to 2018-07-08 and its count is 1056
Now fetching data from 2018-07-09 to 2018-07-12
Got demand data from date 2018-07-09 to 2018-07-12 and its count is 1056
Now fetching data from 2018-07-13 to 2018-07-16
Got demand data from date 2018-07-13 to 2018-07-16 and its count is 1056
Now fetching data from 2018-07-17 to 2018-07-20
Got demand data from date 2018-07-17 to 2018-07-20 and its count is 1056
Now fetching data from 2018-07-21 to 2018-07-24
Got demand data from date 2018-07-21 to 2018-07-24 and its count is 1056
Now fetching data from 2018-07-25 to 2018-07-28
Got demand data from date 2018-07-25 to 2018-07-28 and its count is 1056
Now fetching data from 2018-07-29 to 2018-08-01
Got demand data from date 2018-07-29 to 2018-08-01 and its count is 1056
Now fet

/workspace/VoltWise/.venv/lib/python3.10/site-packages/numpy/lib/nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/workspace/VoltWise/.venv/lib/python3.10/site-packages/numpy/lib/nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)


Data Correction
['2018-11-10', '2018-11-11', '2018-11-12', '2018-11-13']
Got demand data from date 2018-11-10 to 2018-11-13 and its count is 1056
Now fetching data from 2018-11-14 to 2018-11-17
Got demand data from date 2018-11-14 to 2018-11-17 and its count is 1056
Now fetching data from 2018-11-18 to 2018-11-21
Got demand data from date 2018-11-18 to 2018-11-21 and its count is 1056
Now fetching data from 2018-11-22 to 2018-11-25
Got demand data from date 2018-11-22 to 2018-11-25 and its count is 1056
Now fetching data from 2018-11-26 to 2018-11-29
Got demand data from date 2018-11-26 to 2018-11-29 and its count is 1056
Now fetching data from 2018-11-30 to 2018-12-03
Got demand data from date 2018-11-30 to 2018-12-03 and its count is 1056
Now fetching data from 2018-12-04 to 2018-12-07
Got demand data from date 2018-12-04 to 2018-12-07 and its count is 1056
Now fetching data from 2018-12-08 to 2018-12-11
Got demand data from date 2018-12-08 to 2018-12-11 and its count is 1056
Now fet

In [26]:
region_data_dict_copy = dict(region_data_dict)

In [29]:
for k in region_data_dict_copy.keys():
    region_data_dict_copy[k]=pd.concat(region_data_dict_copy[k])

In [31]:
for k in region_data_dict_copy.keys():
    region_data_dict_copy[k].to_csv(f'/workspace/VoltWise/Lambda/Local_code/Data/New_data_csv/{k}-2023-07-10.csv',index=False)
    region_data_dict_copy[k].to_parquet(f'/workspace/VoltWise/Lambda/Local_code/Data/New_data_parquet/{k}.parquet')

In [8]:
ip= '/workspace/VoltWise/Lambda/Local_code/Data/Hourly/New_data_csv'
input_files = os.listdir(ip)
for file in input_files:
    input_path = os.path.join(ip, file)
    output_file = file.split('-')[0]
    df= pd.read_csv(input_path)
    df.drop(columns=['region'],inplace=True)
    print(output_file)
    df.to_csv(f'/workspace/VoltWise/Lambda/Local_code/Data/Hourly/New_data_csv/{output_file}.csv',index=False)
    df.to_parquet(f'/workspace/VoltWise/Lambda/Local_code/Data/Hourly/New_data_parquet/{output_file}.parquet')

CAL
CAR
CENT
FLA
MIDA
MIDW
NE
NY
SE
SW
TEX
